In [2]:
%matplotlib inline

from statsmodels.tools.tools import add_constant
import numpy as np
import pandas as pd
import random, sys, os, warnings
from math import exp
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [9]:
df=pd.read_excel("../input/data2.xlsx")
df["LE > median"]=df["Life expectancy"].apply(lambda x: 1 if (x>70.9905) else 0)

In [10]:
df2 = df[['LE > median','Immunization, measles', 'GDP US$ per capita ln', 'Cereal tn/ha ln']]
df2 = add_constant(df2)
df2 = df2.rename(columns={'Immunization, measles': '1', 'GDP US$ per capita ln': '2', 'Cereal tn/ha ln':'3'})
df_res1 = df2[['const','LE > median']].copy()
df_res2 = df2[['const','LE > median']].copy()
df_resf = df2[['const','LE > median']].copy()

df2.head()

,const,LE > median,1,2,3
0,1.000,1,91.000,7.440,1.044
1,1.000,1,92.000,7.534,0.896
2,1.000,1,95.000,7.424,1.041
3,1.000,1,89.000,7.515,1.055
4,1.000,1,85.000,7.643,1.029


In [5]:
def activation_f(x):
    p = 1 / (1 + exp(-x))
    return (p)

In [ ]:
alpha = 0.01
weights_1 = [[random.uniform(-1,1) for i in range(4)] for j in range(4)] #3 x + const weights for 4 neurons
weights_2 = [[random.uniform(-1,1) for i in range(5)] for j in range(3)] #4 x + const weights for 3 neurons
weights_f = [random.uniform(-1,1) for i in range(4)]                     #3 x + const weights for 1 neuron

for epoch in range(20):
    '''layer 1'''
    for i in range(len(weights_1)):
        df_res1[str(i+1)] = df2['const']*weights_1[i][0] + df2['1']*weights_1[i][1] + df2['2']*weights_1[i][2] + df2['3']*weights_1[i][3]
        df_res1[str(i+1)] = df_res1[str(i+1)].apply(lambda x: activation_f(x))

    '''layer 2'''
    for i in range(len(weights_2)):
        df_res2[str(i+1)] = df_res1['const']*weights_2[i][0] + df_res1['1']*weights_2[i][1] + df_res1['2']*weights_2[i][2] + df_res1['3']*weights_2[i][3] + df_res1['4']*weights_2[i][4]
        df_res2[str(i+1)] = df_res2[str(i+1)].apply(lambda x: activation_f(x))

    '''output'''
    df_resf['predicted'] = df_res2['const']*weights_f[0] + df_res2['1']*weights_f[1] + df_res2['2']*weights_f[2] + df_res2['3']*weights_f[3]
    df_resf['predicted'] = df_resf['predicted'].apply(lambda x: activation_f(x))
    df_resf['err'] = df_resf['LE > median'] - df_resf['predicted']
    
    result = (df_resf['err']*df_resf['err']).sum()
    print(result)

    '''calculate values to update weights_f'''
    df_resf['delta'] = -1* df_resf['err'] * df_resf['predicted'] * (1 - df_resf['predicted'])

    '''bp to layer 2'''
    for i in range(len(weights_2)):
        df_res2['delta'+str(i+1)] = df_res2[str(i+1)] * (1 - df_res2[str(i+1)]) * df_resf['delta'] * weights_f[i+1]
        

    '''bp to layer 1'''
    for i in range(len(weights_1)):
        df_res1['delta'+str(i+1)] = df_res1[str(i+1)] * (1 - df_res1[str(i+1)]) * (df_res2['delta1'] * weights_2[0][i+1] + df_res2['delta2'] * weights_2[1][i+1] + df_res2['delta3'] * weights_2[2][i+1])


    '''update w'''
    for i in range(len(weights_f)-1):
        df_resf['grad'+str(i+1)] = df_resf['delta'] * df_res2[str(i+1)]
        weights_f[i+1] -= alpha * df_resf['grad'+str(i+1)].sum()

    for i in range(len(weights_2)):
        for j in range(4):
            df_res2['grad'] = df_res2['delta'+str(i+1)] * df_res1[str(j+1)]
            weights_2[i][j+1] -= alpha * df_res2['grad'].sum()
    for i in range(len(weights_1)):
        for j in range(3):
            df_res1['grad'] = df_res1['delta'+str(i+1)] * df2[str(j+1)]
            weights_1[i][j+1] -= alpha * df_res1['grad'].sum()

800.3441449576421
816.5820851014265
759.8452077113906
742.8225979284857
723.4018808009594


In [15]:
df_resf

,const,LE > median,predicted,err,delta,grad1,grad2,grad3
0,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
1,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
2,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
3,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
4,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
5,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
6,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
7,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
8,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133
9,1.000,1,0.456,0.544,-0.135,-0.130,-0.126,-0.133


In [ ]:
df_res2.head(3)

In [ ]:
df_res1.head(3)